# Preliminaries
We need to load a few libraries first

In [3]:
import pandas as pd
import numpy as np
import ampligraph
import stardog
import json
import io
print(ampligraph.__version__)

2.0.0


# Loading the data

In [4]:
df = pd.read_csv("graph_content.csv.gz").dropna()
df.head()

,s,p,o
0,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8472
1,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9825
2,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:10260
3,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8668
4,tag:stardog:designer:Tutorial:data:Match:3251,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9850


# Train the model

In [5]:
from ampligraph.evaluation import train_test_split_no_unseen 
test_size = int(df.shape[0]*0.10)
X_train, X_valid = train_test_split_no_unseen(np.array(df), test_size=test_size)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (9090, 3)
Test set size:  (1010, 3)


In [6]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.regularizers import get as get_regularizer
from ampligraph.latent_features.loss_functions import get as get_loss
import tensorflow as tf

# create the model
model = ScoringBasedEmbeddingModel(k=100, eta=10,
                                   scoring_type='ComplEx', seed=0)

# create reg, loss, optim
reg = get_regularizer('LP', hyperparams={'p':3, 'lambda':1e-3})
loss = get_loss('pairwise', hyperparams={'margin':5})
optim = tf.keras.optimizers.Adam(learning_rate = 1e-4)

# compile the model
model.compile(loss=loss, optimizer=optim, entity_relation_regularizer=reg)
model.fit(X_train, batch_size=5000, epochs=500)

Metal device set to: Apple M1 Pro
WARNING - At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
Epoch 1/500


2023-05-06 07:03:48.219105: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 4s 1s/step - loss: 227250.4688
Epoch 2/500
3/3 [==============================] - 0s 98ms/step - loss: 227249.2500
Epoch 3/500
3/3 [==============================] - 0s 103ms/step - loss: 227248.0469
Epoch 4/500
3/3 [==============================] - 0s 96ms/step - loss: 227246.9062
Epoch 5/500
3/3 [==============================] - 0s 92ms/step - loss: 227245.7969
Epoch 6/500
3/3 [==============================] - 0s 94ms/step - loss: 227244.6875
Epoch 7/500
3/3 [==============================] - 0s 97ms/step - loss: 227243.5000
Epoch 8/500
3/3 [==============================] - 0s 99ms/step - loss: 227242.3125
Epoch 9/500
3/3 [==============================] - 0s 116ms/step - loss: 227241.0938
Epoch 10/500
3/3 [==============================] - 0s 94ms/step - loss: 227239.9062
Epoch 11/500
3/3 [==============================] - 0s 97ms/step - loss: 227238.6875
Epoch 12/500
3/3 [==============================] - 0s 97ms/step - loss: 227237.4219
E

In [7]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
ranks = model.evaluate(X_valid, batch_size=2000, verbose=True)
print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('Hits@1:', hits_at_n_score(ranks, 1))
print('Hits@3:', hits_at_n_score(ranks, 3))
print('Hits@10:', hits_at_n_score(ranks, 10))
print('Hits@100:', hits_at_n_score(ranks, 100))

2023-05-06 07:06:44.443297: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/RowPartitionFromRowLengths/assert_non_negative/assert_less_equal/Assert/AssertGuard/branch_executed/_8


2/2 [==============================] - 2s 1s/step
MR: 733.450495049505
MRR: 0.07365019666112425
Hits@1: 0.0
Hits@3: 0.10247524752475247
Hits@10: 0.18613861386138614
Hits@100: 0.3613861386138614


# Make predictions

We will now generate all the pairs of <player, plays for, team> and rank them by prediction score

In [8]:
players = set(df[df['s'].str.contains('Player:')]['s'].values) | \
          set(df[df['o'].str.contains('Player:')]['o'].values)
teams = set(df[df['s'].str.contains('Team:')]['s'].values) | \
        set(df[df['o'].str.contains('Team:')]['o'].values)
print('Number of players: ', len(players))
print('Number of teams: ', len(teams))

Number of players:  2344
Number of teams:  98


In [9]:
pairs = []
for player in players:
  for team in teams:
    pairs.append([player, 'tag:stardog:designer:Tutorial:model:plays_for', team])
# Create the possible combinations to test
df_to_test = pd.DataFrame(pairs, columns=['s', 'p', 'o'])
# Remove all the pairs we used for training
df_to_test = pd.merge(df_to_test, df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
print (df_to_test.shape)
df_to_test.head()

(227262, 3)


,s,p,o
0,tag:stardog:designer:Tutorial:data:Player:96645,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8551
1,tag:stardog:designer:Tutorial:data:Player:96645,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8661
2,tag:stardog:designer:Tutorial:data:Player:96645,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8226
3,tag:stardog:designer:Tutorial:data:Player:96645,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:10242
4,tag:stardog:designer:Tutorial:data:Player:96645,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9906


In [10]:
df_to_test['scores'] = model.predict(np.array(df_to_test))
df_to_test = df_to_test.sort_values(by='scores', ascending=False)
df_to_test

,s,p,o,scores
189014,tag:stardog:designer:Tutorial:data:Player:49860,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9885,1.099297
228227,tag:stardog:designer:Tutorial:data:Player:163670,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:10269,1.056272
104425,tag:stardog:designer:Tutorial:data:Player:27420,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8721,1.053218
50344,tag:stardog:designer:Tutorial:data:Player:24504,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9885,1.032938
201460,tag:stardog:designer:Tutorial:data:Player:39535,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9885,1.029492
...,...,...,...,...
70614,tag:stardog:designer:Tutorial:data:Player:24970,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9798,-0.534542
98014,tag:stardog:designer:Tutorial:data:Player:25597,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9837,-0.537461
157360,tag:stardog:designer:Tutorial:data:Player:42304,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9885,-0.539545
149253,tag:stardog:designer:Tutorial:data:Player:299653,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8455,-0.543965


In [11]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981']

,s,p,o,scores
86849,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:10267,0.784811
86920,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8560,0.691204
86906,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:7878,0.672287
86881,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8315,0.658919
86923,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8302,0.651968
...,...,...,...,...
86921,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8686,-0.205425
86853,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8659,-0.222288
86828,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8551,-0.270590
86844,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8528,-0.308764


# Improve the presentation
Figuring things out based on URIs is a bit challenging. We will return to Stardog to ask for the label associated to each of those identifiers and update the data frame with those.

For running the following cells you will need to create a file `connection_details.json` with this content in it:

```json
{
  "database": "YOUR DATABASE (e.g. Tutorial)",
  "endpoint": "YOUR ENDPOINT (e.g. https://sd-0fcedc52.stardog.cloud:5820)",
  "username": "YOUR USERNAME (e.g. christophe.gueret@accenture.com)",
  "password": "YOUR PASSWORD (e.g. Passw0rd)",
}
```

In [12]:
connection_details = json.load(open('connection_details.json'))
conn = stardog.Connection(**connection_details)

In [13]:
query = """
SELECT ?identifier ?label FROM stardog:context:all WHERE {
  ?identifier rdfs:label ?label .
}
"""
csv_results = conn.select(query, content_type='text/csv')
df_labels = pd.read_csv(io.BytesIO(csv_results)).set_index('identifier')
labels = df_labels.to_dict()['label']
print ('Loaded {} labels'.format(len(labels)))

ConnectionError: HTTPSConnectionPool(host='sd-0fcedc52.stardog.cloud', port=5820): Max retries exceeded with url: /Tutorial/query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x29d99b070>: Failed to resolve 'sd-0fcedc52.stardog.cloud' ([Errno 8] nodename nor servname provided, or not known)"))

In [91]:
df_to_test.replace(labels)

,s,p,o,scores
65209,Lionel Messi,plays for,Real Madrid CF,0.511274
229018,Federico Peluso,plays for,Palermo,0.462421
88682,Emanuele Calaio,plays for,Palermo,0.437658
49507,Cristiano Ronaldo,plays for,FC Barcelona,0.406167
199485,Jacques Alaixys Romao,plays for,Toulouse FC,0.387238
...,...,...,...,...
187125,Andre Auras,plays for,Hamburger SV,-0.194937
72095,Timothee Dieng,plays for,CA Osasuna,-0.204001
213095,Jordan Bowery,plays for,Hamburger SV,-0.210471
213108,Jordan Bowery,plays for,Athletic Club de Bilbao,-0.222462


In [93]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981'].replace(labels)

,s,p,o,scores
65209,Lionel Messi,plays for,Real Madrid CF,0.511274
65190,Lionel Messi,plays for,Atlético Madrid,0.281800
65183,Lionel Messi,plays for,Granada CF,0.279305
65200,Lionel Messi,plays for,RC Celta de Vigo,0.264320
65226,Lionel Messi,plays for,Athletic Club de Bilbao,0.254224
...,...,...,...,...
65185,Lionel Messi,plays for,Inter,-0.077301
65265,Lionel Messi,plays for,Girondins de Bordeaux,-0.080506
65248,Lionel Messi,plays for,Stade Rennais FC,-0.092022
65233,Lionel Messi,plays for,Arsenal,-0.107957
